In [1]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd

In [9]:
load_dotenv()

True

In [11]:
OPENAI_API_KEY=os.getenv("GenAI")

In [12]:
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [13]:
embedding = OpenAIEmbeddings()

In [15]:
ASTRA_DB_API_ENDPOINT="https://d5da7470-4035-4620-963e-f9397ebe6f95-us-east-2.apps.astra.datastax.com"

In [16]:
ASTRA_DB_APPLICATION_TOKEN="AstraCS:YrTrChclmIQCZOyaFgBGEKpf:3bebb5b9c99320e3314069921abcd3ed6697bf50f63482828a4700874e8c1966"

In [17]:
ASTRA_DB_KEYSPACE= "default_keyspace"

In [18]:
collection_name= "financebot"

In [19]:
vstore = AstraDBVectorStore(
    embedding= embedding,
    collection_name= collection_name,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE
    )

In [20]:
# do pip install langchain_community in cmd prompt
from langchain_community.document_loaders import PyPDFLoader

ModuleNotFoundError: No module named 'langchain_community'

In [35]:
loader = PyPDFLoader("C:\\Complete_Content\\GENERATIVEAI\\llmopstradingbot\\data\\finance_data.pdf")

NameError: name 'PyPDFLoader' is not defined

In [ ]:
pages = loader.load()

In [ ]:
len(pages)

In [ ]:
pages=pages[10:20]

In [ ]:
pages[0].page_content

In [ ]:
for i, doc in enumerate(pages):
    print(i , doc)

In [ ]:
raw_text = ''
for i, doc in enumerate(pages):
    text = doc.page_content
    if text:
        raw_text += text

In [ ]:
print(raw_text)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
)

In [ ]:
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

In [ ]:
texts[0]

In [ ]:
from langchain_core.documents import Document

In [ ]:
docs = []
for i in range(len(texts)):
    doc = Document(page_content=texts[i])
    docs.append(doc)

In [ ]:
docs

In [ ]:
vstore.add_documents(docs)

In [ ]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
retriever.get_relevant_documents("what is Market For Registrant’s Common Equity?")

In [ ]:
PRODUCT_BOT_TEMPLATE = """
    Your finance bot is an expert in finance related advice.
    Ensure your answers are relevant to the query context and refrain from straying off-topic.
    Your responses should be concise and informative.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER:
    
    """

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template(PRODUCT_BOT_TEMPLATE)

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI()

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

In [ ]:
chain.invoke("what is Market For Registrant’s Common Equity?")